In [100]:
import torch

device = torch.device("cuda:4")

In [101]:
from spuco.utils import set_seed

set_seed(0)

In [102]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="../data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train",
    label_noise=0
)
trainset.initialize()

testset = SpuCoMNIST(
    root="../data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:01<00:00, 5586.58it/s]


In [103]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="../data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()

100%|██████████| 11996/11996 [00:02<00:00, 4708.61it/s]


In [104]:
T.ToPILImage()(trainset[10732][0]).resize((28,28))

## Implement the Dataloader

In [105]:
from torch.utils.data import DataLoader, Dataset

class EnDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        return x, y, idx

train_dataloader = DataLoader(EnDataset(trainset), batch_size=128, shuffle=True, num_workers=4)
val_dataloader = DataLoader(EnDataset(valset), batch_size=128, shuffle=False, num_workers=4)
test_dataloader = DataLoader(EnDataset(testset), batch_size=128, shuffle=False, num_workers=4)

# Setting the Hyperparameter

In [106]:
epochs =10
early_stopping = 10

# Implement the robust_model and Optimizer

In [107]:
from spuco.models import model_factory
from torch.optim import Adam
model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Trainning Process

In [108]:
from tqdm import tqdm
import numpy as np

# Calculate the forgetting score during training
prev_acc = np.zeros(len(trainset))
forgetting = np.zeros(len(trainset))

kill_cnt = 0
best_acc = 0.0

for epoch in range(epochs):
    model.train()
    with tqdm(total=len(train_dataloader), dynamic_ncols=True) as t:
        t.set_description(f"Epoch {epoch+1}/{epochs}")
        for x, y, idx in train_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            loss = torch.nn.functional.cross_entropy(y_hat, y)
            acc = (y_hat.argmax(dim=1) == y).float()

            # Calculate forgetting
            for batch_idx, i in enumerate(idx):
                if prev_acc[i] > acc[batch_idx]:
                    forgetting[i] = forgetting[i]+1
            
            prev_acc[idx] = acc.cpu().numpy()
            acc = acc.mean()

            loss.backward()
            optimizer.step()

            t.set_postfix({
                "loss": loss.item(),
                "acc": acc.item()
            })
            t.update()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y, idx in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            correct += (y_hat.argmax(dim=1) == y).sum().item()
            total += y.size(0)
    valid_acc = correct/total
    print(f"Validation accuracy: {valid_acc}")
    if valid_acc > best_acc:
        best_acc = valid_acc
        best_epoch = epoch + 1
        kill_cnt = 0

        torch.save(model.state_dict(), "output/first_model.pth")
        print("Saving model...")
    else:
        kill_cnt += 1
        if kill_cnt == early_stopping:
            print(f"Early stopping at epoch {epoch+1}")
            break
    

model.eval()
model.load_state_dict(torch.load("output/first_model.pth"))
correct = 0
total = 0
with torch.no_grad():
    for x, y, idx in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_hat = model(x)
        correct += (y_hat.argmax(dim=1) == y).sum().item()
        total += y.size(0)
print(f"Best epoch: {epoch + 1}, best validation accuracy: {best_acc}")
print(f"Test accuracy: {correct/total}")
        

Epoch 1/10: 100%|██████████| 376/376 [00:05<00:00, 66.50it/s, loss=0.0106, acc=1]    


Validation accuracy: 0.1999833277759253
Saving model...


Epoch 2/10: 100%|██████████| 376/376 [00:05<00:00, 70.52it/s, loss=0.0141, acc=1]    


Validation accuracy: 0.1999833277759253


Epoch 3/10: 100%|██████████| 376/376 [00:05<00:00, 73.31it/s, loss=0.00226, acc=1]   


...

Validation accuracy: 0.5260920306768923


Epoch 10/10: 100%|██████████| 376/376 [00:05<00:00, 72.96it/s, loss=0.000904, acc=1]   


Validation accuracy: 0.6242914304768256
Saving model...


/tmp/ipykernel_2484383/2809828233.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("output/first_model.pth"))


Best epoch: 10, best validation accuracy: 0.6242914304768256
Test accuracy: 0.637


# Calculating wrong index

In [109]:
model.eval()
wrong_idx = []
for x, y, idx in train_dataloader:
    x, y, idx = x.to(device), y.to(device), idx.to(device)
    y_hat = model(x)
    wrong_batch_idx = (y_hat.argmax(dim=1) != y).nonzero()
    wrong_idx.append(idx[wrong_batch_idx].cpu().numpy())

wrong_idx = np.concatenate(wrong_idx)


In [110]:
print(len(np.nonzero(forgetting[wrong_idx])[0]))
print(len(wrong_idx))
print(forgetting[wrong_idx].mean())
print(forgetting[wrong_idx].max())
print(forgetting[wrong_idx].min())

39
164
0.29878048780487804
3.0
0.0


# JTT, Choose from 1,2,3,4 to run

In [111]:
upsample_factor = 100
from torch.utils.data import Subset

## 1. Original JTT

In [112]:
upsample_idx = np.concat((np.arange(len(trainset)), np.repeat(wrong_idx, upsample_factor)))
upsample_trainset = Subset(trainset, upsample_idx)

## 2. Forgrtting Scores

In [82]:
wrong_sample_fgscore = forgetting[wrong_idx]
total_repeats = upsample_factor * len(wrong_idx)
initial_repeats = wrong_sample_fgscore / wrong_sample_fgscore.sum() * total_repeats
repeats = np.round(initial_repeats).astype(int)

while repeats.sum() != total_repeats:
    diff = total_repeats - repeats.sum()
    if diff > 0:
        repeats[np.argmax(initial_repeats - repeats)] += 1
    else:
        repeats[np.argmax(repeats - initial_repeats)] -= 1

repeats = repeats.squeeze()
upsample_idx = np.concat((np.arange(len(trainset)), np.repeat(wrong_idx, repeats)))
upsample_trainset = Subset(trainset, upsample_idx)

## 3. Modified Forgetting Scores (Replace 0 forgetting score as max + 1)

In [88]:
wrong_sample_fgscore = forgetting[wrong_idx]
max_fgscore = wrong_sample_fgscore.max()
wrong_sample_fgscore[wrong_sample_fgscore == 0] = max_fgscore

total_repeats = upsample_factor * len(wrong_idx)
initial_repeats = wrong_sample_fgscore / wrong_sample_fgscore.sum() * total_repeats
repeats = np.round(initial_repeats).astype(int)

while repeats.sum() != total_repeats:
    diff = total_repeats - repeats.sum()
    if diff > 0:
        repeats[np.argmax(initial_repeats - repeats)] += 1
    else:
        repeats[np.argmax(repeats - initial_repeats)] -= 1

repeats = repeats.squeeze()
upsample_idx = np.concat((np.arange(len(trainset)), np.repeat(wrong_idx, repeats)))
upsample_trainset = Subset(trainset, upsample_idx)

## 4. Modified Forgetting Scores (Add 1 to all forgetting scores)

In [94]:
wrong_sample_fgscore = forgetting[wrong_idx]+1
total_repeats = upsample_factor * len(wrong_idx)
initial_repeats = wrong_sample_fgscore / wrong_sample_fgscore.sum() * total_repeats
repeats = np.round(initial_repeats).astype(int)

while repeats.sum() != total_repeats:
    diff = total_repeats - repeats.sum()
    if diff > 0:
        repeats[np.argmax(initial_repeats - repeats)] += 1
    else:
        repeats[np.argmax(repeats - initial_repeats)] -= 1

repeats = repeats.squeeze()
upsample_idx = np.concat((np.arange(len(trainset)), np.repeat(wrong_idx, repeats)))
upsample_trainset = Subset(trainset, upsample_idx)

In [113]:
print(repeats)

[ 74  74  74  74  74 147  74  74 147 294  74  74 147 147  74  74 147  74
  74  74  74 147  74  74  74  74  74  74 147  74  74  74 147  74  74  74
  74  74  74  74  74  74  74  74  74  74 147  74  74  74  74 147  74  74
  74  74  74  74  74  74  74  74 147 147 220  74  74  74 147  74  74  74
  74  74  74 147  74 147 147  73 147  73 220  73  73  73 147 147 147 147
  73 147  73  73  73  73  73 220  73  73 147  73  73  73  73 147  73  73
  73  73  73 147  73  73  73 147  73 147 147  73  73  73 220  73 147  73
 147  73  73  73 147 147 147  73 147  73  73  73  73  73  73  73  73 147
 220 220  73  73  73  73  73  73  73  73  73  73  73  73  73  73  73 147
 294  73  73  73 147  73  73  73  73  73  73  73  73 147 147 220 147  73
  73  73 147  73  73  73  73  73  73 220 220  73  73 147 147  73  73  73
 147  73  73  73  73  73  73 147  73 147 147  73  73  73  73 220  73 147
  73  73 220  73  73 147  73  73  73 220  73 147 147 147 147  73 220  73
  73  73  73  73  73  73  73  73  73 147  73 220  7

# Setting Hyperparameter

In [114]:
epochs = 50
early_stopping = 4

# Training

In [115]:
train_dataloader = DataLoader(EnDataset(upsample_trainset), batch_size=128, shuffle=True, num_workers=4)

In [116]:
robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
optimizer = Adam(robust_model.parameters(), lr=1e-3, weight_decay=1e-4)

In [117]:
from tqdm import tqdm
import numpy as np

kill_cnt = 0
best_acc = 0.0
for epoch in range(epochs):
    robust_model.train()
    with tqdm(total=len(train_dataloader), dynamic_ncols=True) as t:
        t.set_description(f"Epoch {epoch+1}/{epochs}")
        for x, y, idx in train_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = robust_model(x)
            loss = torch.nn.functional.cross_entropy(y_hat, y)
            acc = (y_hat.argmax(dim=1) == y).float()
            acc = acc.mean()

            loss.backward()
            optimizer.step()

            t.set_postfix({
                "loss": loss.item(),
                "acc": acc.item()
            })
            t.update()

    robust_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y, idx in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_hat = robust_model(x)
            correct += (y_hat.argmax(dim=1) == y).sum().item()
            total += y.size(0)
    valid_acc = correct/total
    print(f"Validation accuracy: {valid_acc}")
    if valid_acc > best_acc:
        best_acc = valid_acc
        best_epoch = epoch + 1
        kill_cnt = 0

        torch.save(robust_model.state_dict(), "output/robust_model.pth")
        print("Saving model...")
    else:
        kill_cnt += 1
        if kill_cnt == early_stopping:
            print(f"Early stopping at epoch {epoch+1}")
            break


robust_model.eval()
robust_model.load_state_dict(torch.load("output/robust_model.pth"))
correct = 0
total = 0
with torch.no_grad():
    for x, y, idx in test_dataloader:
        x, y = x.to(device), y.to(device)
        y_hat = robust_model(x)
        correct += (y_hat.argmax(dim=1) == y).sum().item()
        total += y.size(0)
print(f"Best epoch: {epoch + 1}, best validation accuracy: {best_acc}")
print(f"Test accuracy: {correct/total}")

Epoch 1/50: 100%|██████████| 504/504 [00:03<00:00, 149.72it/s, loss=0.187, acc=1]    


Validation accuracy: 0.3911303767922641
Saving model...


Epoch 2/50: 100%|██████████| 504/504 [00:03<00:00, 158.62it/s, loss=0.178, acc=0.95]  


Validation accuracy: 0.5067522507502501
Saving model...


Epoch 3/50: 100%|██████████| 504/504 [00:03<00:00, 151.10it/s, loss=0.168, acc=0.95]  


...

Validation accuracy: 0.660053351117039


Epoch 13/50: 100%|██████████| 504/504 [00:03<00:00, 156.04it/s, loss=0.000654, acc=1]   


Validation accuracy: 0.6707235745248417
Early stopping at epoch 13


/tmp/ipykernel_2484383/3517860651.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  robust_model.load_state_dict(torch.load("output/robust_model.pth"))


Best epoch: 13, best validation accuracy: 0.6941480493497832
Test accuracy: 0.7029
